# 라이브러리 추가

In [1]:
import pandas as pd
import os
import sys
import json
from collections import Counter

# 파일 읽어오기   
* 파일 경로를 가져온다.
    * 만약 google drive를 통해 파일을 가져온다면 아래와 같이 파일 경로를 찾아 오자.
    * `!find /content/drive/MyDrive/ -name read/*`

In [ ]:
file_path = input("file path plz :)(ex) /content/drive/MyDrive/kakao arena/data/read)>>")

# .2019010120_2019010121.un~ 삭제

# read 디렉터리 내의 파일 리스트를 가지고 옵니다.
read_file_list = os.listdir(file_path)  
read_list = []  # file_name : 파일명
                # user_id : user id
                # content_id : 읽은 글 id

for file_name in read_file_list:
    tmp_path = os.path.join(file_path, file_name)
    with open(tmp_path, 'r') as f:
        for line in f:
            line_list = line.split()
            read_list.append({"file_name":file_name,"user_id":line_list[0],"content_id":line_list[1:]})

read_list = pd.DataFrame(read_list)

read_list.to_pickle(file_path + ".pkl")

read_list.head(5)

,file_name,user_id,content_id
0,2019012301_2019012302,#6b77f7aadb7ea35aa91ec34987ac0b5a,[@pinastro_6]
1,2019012301_2019012302,#fdde4774182dfe1f8f87aa976d94bab0,"[@dailylife_666, @munchi_78, @haereka_93, @sun..."
2,2019012301_2019012302,#859958bb4e89736c4a98fc861e15d691,"[@brunchtq1f_42, @iswim_12, @nrlee0610_16]"
3,2019012301_2019012302,#1f6c9f5a0e4dc284a142212bce672940,"[@ggoforit_46, @tenbody_1631, @sweetannie_140,..."
4,2019012301_2019012302,#afb958144010a82523a266f413432e80,"[@bookerbuker_267, @bookerbuker_267, @bookerbu..."


* read 디렉토리: 본 글 정보

  * read.tar은 2018년 10월 1일부터 2019년 3월 1일까지 일부 브런치 독자들이 본 글의 정보가 총 3,625개의 파일로 구성되어 있습니다.
  * 파일의 이름은 시작일_종료일 형태입니다. 예를 들어 2018110708_2018110709 파일은 2018년 11월 7일 오전 8시부터 2018년 11월 7일 오전 9시 전까지 본 글입니다.
  * 파일은 여러 줄로 이뤄져 있으며 하나의 줄은 브런치의 독자가 파일의 시간 동안 본 글을 시간 순으로 기록한 것입니다. 한 줄의 정보는 공백으로 구분되어 있으며 첫번 째가 독자의 암호화된 식별자이고 그 뒤로는 해당 독자가 본 글의 정보입니다.
  * 예를 들어 read/2019022823_2019030100 파일에 기록된 아래 정보는 #8a706ac921a11004bab941d22323efab 라는 독자가 2019년 2월 28일 23시에서 2019년 3월 1일 0시 사이에 @bakchacruz_34 @wo-motivator_133 @wo-motivator_133 를 순서대로 보았다는 뜻입니다. @wo-motivator_133 글이 두 번 나타난 것은 이 글을 보기 위해 두 번 방문했다는 뜻입니다.
    * #8a706ac921a11004bab941d22323efab @bakchacruz_34 @wo-motivator_133 @wo-motivator_133
  * 글을 보았다는 의미는 특정 글에 모바일, PC, 앱을 통해 접근했다는 뜻입니다. 머문 시간에 대한 정보가 제공되지 않기 때문에 실제로 글을 읽지 않고 이탈하는 등의 가능성도 있습니다.

# dev/test, train/test split

*   **dev/test** : 개발 데이터 3000명과 평가 데이터 5000명으로 분리
*   **train/test** : 2019년 2월 22일 이전과 이후로 분리

|  | dev | test |
|:-:|:-:|:-:|
| **train** | 2019년 2월 22일 이전, 3000명 | 2019년 2월 22일 이전, 5000명 |
| **test** | 2019년 2월 22일 이후, 3000명 | 2019년 2월 22일 이후, 5000명 |

In [2]:
file_path = input("file path plz :)(ex) /content/drive/MyDrive/kakao arena/data/)>>")

read_data = pd.read_pickle(file_path+"read.pkl")

read_data.head()

file path plz :)(ex) /content/drive/MyDrive/kakao arena/data/)>>/content/drive/My Drive/한양대학교/브런치 사용자를 위한 글 추천 대회/data/


,file_name,user_id,content_id
0,2019012301_2019012302,#6b77f7aadb7ea35aa91ec34987ac0b5a,[@pinastro_6]
1,2019012301_2019012302,#fdde4774182dfe1f8f87aa976d94bab0,"[@dailylife_666, @munchi_78, @haereka_93, @sun..."
2,2019012301_2019012302,#859958bb4e89736c4a98fc861e15d691,"[@brunchtq1f_42, @iswim_12, @nrlee0610_16]"
3,2019012301_2019012302,#1f6c9f5a0e4dc284a142212bce672940,"[@ggoforit_46, @tenbody_1631, @sweetannie_140,..."
4,2019012301_2019012302,#afb958144010a82523a266f413432e80,"[@bookerbuker_267, @bookerbuker_267, @bookerbu..."


In [3]:
read_data["tmp_file_name"] = read_data["file_name"].apply(lambda x : int(x[:8]))

# 2019년 02월 22일 00시를 기준으로 train/test로 구분
read_train_data = read_data[read_data["tmp_file_name"] < 20190222]
read_test_data = read_data[read_data["tmp_file_name"] >= 20190222]

In [4]:
dev_users_data = []

with open(file_path+"predict/dev.users", 'r') as f:
    for line in f:
        line = line.strip()
        dev_users_data.append(line)

dev_users_data = pd.DataFrame({"user_id":dev_users_data})

read_dev_users_train_data = pd.merge(read_train_data, dev_users_data, left_on='user_id', right_on='user_id', how='inner')
read_dev_users_train_data = read_dev_users_train_data.groupby("user_id").apply(lambda x : [element for array in x["content_id"].values for element in array])

print("dev_train_data :",read_dev_users_train_data)

read_dev_users_test_data = pd.merge(read_test_data, dev_users_data, left_on='user_id', right_on='user_id', how='inner')
read_dev_users_test_data = read_dev_users_test_data.groupby("user_id").apply(lambda x : [element for array in x["content_id"].values for element in array])

print("dev_test_data :",read_dev_users_test_data)

dev_train_data : user_id
#000fe61478d384d09f3bcdd0c2f5227d    [@ddamimovie_1485, @ddamimovie_1485, @ddamimov...
#0029e554ab774e79b69ee9217dee06db    [@binkond_1183, @readream3_25, @mothertive_43,...
#005f71b15b9423f335b85626ceec86a6    [@minigorae_248, @minigorae_250, @jiholim_82, ...
#0085e940b6e94be6efe4e275d1966615    [@sssfriend_52, @gracefull_22, @prestigegorill...
#008d4d3ae2f49a95f9d82d9e1d8a97ac    [@klida8866_32, @klida8866_32, @klida8866_33, ...
                                                           ...                        
#ffab618a20ff46fdac7bb842b28f005d    [@holidaymemories_139, @jordan777_2515, @jorda...
#ffae495f7c65c0ab7cb4eac72e84bc20    [@the8work_78, @the8work_78, @the8work_80, @th...
#ffb3a1e1514e9afa92914cc1a74e4ce6    [@babytone2_14, @lucete0710_3, @12myapril_149,...
#ffeabb5aaf948bcc15be2f88ebeab3c4    [@moonwalker_7, @moonwalker_7, @moonwalker_7, ...
#fffca5f83f8b7145b3102aec2e0934dc    [@aou_202, @libraryman_137, @sy876_20, @sy876_...
Length: 2710, dtyp

In [5]:
test_users_data = []

with open(file_path+"predict/test.users", 'r') as f:
    for line in f:
        line = line.strip()
        test_users_data.append(line)

test_users_data = pd.DataFrame({"user_id":test_users_data})

read_test_users_train_data = pd.merge(read_train_data, test_users_data, left_on='user_id', right_on='user_id', how='inner')
read_test_users_train_data = read_test_users_train_data.groupby("user_id").apply(lambda x : [element for array in x["content_id"].values for element in array])

print("test_train_data :",read_test_users_train_data)

read_test_users_test_data = pd.merge(read_test_data, test_users_data, left_on='user_id', right_on='user_id', how='inner')
read_test_users_test_data = read_test_users_test_data.groupby("user_id").apply(lambda x : [element for array in x["content_id"].values for element in array])

print("test_test_data :",read_test_users_test_data)

test_train_data : user_id
#000474bba0c00c70e12ac7cfc3d04553    [@insight-kmh_18, @fastfive_63, @fastfive_62, ...
#000c9b7827aba665e4c957f4223629fc    [@onsenwhale_45, @klee0807_49, @binkond_1158, ...
#000fe61478d384d09f3bcdd0c2f5227d    [@ddamimovie_1485, @ddamimovie_1485, @ddamimov...
#00222a855547061a7fd00e404cb96148    [@hukho_235, @hukho_235, @choyh0646_448, @choy...
#0029e554ab774e79b69ee9217dee06db    [@binkond_1183, @readream3_25, @mothertive_43,...
                                                           ...                        
#ffcf2ddfd60422c7b487fa737dce046f    [@onyouhe_65, @onyouhe_65, @soccumi_84, @kosy0...
#ffd6bc8f8d447cc6245c2ef2c4428e21    [@pomnyun_100, @hibikie_9, @seochogirl_2, @han...
#ffeabb5aaf948bcc15be2f88ebeab3c4    [@moonwalker_7, @moonwalker_7, @moonwalker_7, ...
#fffca5f83f8b7145b3102aec2e0934dc    [@aou_202, @libraryman_137, @sy876_20, @sy876_...
#fffcc9c67c1dfb2543eccdde30c19d6f    [@vigorous21_558, @vigorous21_557, @dongyeonse...
Length: 4496, dty

# 조회수

In [6]:
content = pd.DataFrame(Counter([j for i in read_data["content_id"].tolist() for j in i]).most_common())

content.head(5)

,0,1
0,@brunch_141,97206
1,@brunch_151,66903
2,@brunch_145,31162
3,@tenbody_1305,28922
4,@intlovesong_28,23760
